In [1]:
import pandas as pd
import numpy as np
import warnings
import sys

sys.path.append('..')
from modules.utils import make_junction_table, split_rows, get_cassete_exons_from_ipsa, make_track
from pathlib import Path

**Specify the paths and which files we take, and also which samples they correspond to**

_Example:_ for mice RNA-seq experiments

In [ ]:
path = Path('./pyIPSA_mice/').expanduser() # Path to raw pyIPSA data
path_out_bed = path/'cassete_exons_mice.bed' # Path to custom track
path_out_csv = path/'cassete_exons_mice.csv' # Path to table with cassete exons

meta_mice = pd.read_csv(path/'SraRunTable.csv')
meta_mice = meta_mice[['Run', 'treatment']]
meta_mice = meta_mice.replace({'Harringtonine': 'ctrl_chx', 'untreated': 'ctrl', 'Cycloheximide': 'ctrl_chx'})
meta_mice.rename(columns={"Run": "sample", "treatment": "group"}, inplace=True)

meta_junctions = meta_mice.copy()
meta_sites = meta_mice.copy()
meta_junctions['files'] = meta_junctions['sample'].astype(str) + '_Aligned.sortedByCoord.out.J6'
meta_sites['files'] = meta_sites['sample'].astype(str) + '_Aligned.sortedByCoord.out.S6'

Collect a table of all experiments for splice junctions and sites

In [4]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)

    colnames_j = ['junction_id', 'total_count', 'staggered_count', 'entropy', 'annotation_status', 'splice_site']
    data_j6 = make_junction_table(meta_junctions, path, colnames_j)

    colnames_s6 = ['site_id', 'total_count', 'staggered_count', 'entropy']
    data_s6 = make_junction_table(meta_sites, path, colnames_s6)

Merge J6 and S6 tables first by start in S6, then by end in S6

In [5]:
data_merged = data_j6.merge(data_s6.drop('strand', axis=1), left_on=['experiment', 'chrom', 'start'], right_on=['experiment', 'chrom', 'site'], how='left', suffixes=('', '_start'))
data_merged = data_merged.merge(data_s6.drop('strand', axis=1), left_on=['experiment', 'chrom', 'end'], right_on=['experiment', 'chrom', 'site'], how='left', suffixes=('', '_end'))

Check that we have several reads with new sites

In [6]:
# Apply the function and expand the list of rows in the DataFrame
data_merged_expanded = pd.DataFrame([item for sublist in data_merged.apply(split_rows, axis=1) for item in sublist])

data_merged_expanded = data_merged_expanded.drop(columns=['total_count_start', 'site_id_end', 'total_count_end', 'site_end'])
data_merged_expanded

,junction_id,experiment,total_count,annotation_status,chrom,start,end,strand,site_id,site,total_count_site
0,chr10_100584009_100586191_-,ctrl_chx,5,3,chr10,100584009,100586191,-,NaN,NaN,NaN
1,chr10_105763461_105765220_-,ctrl_chx,17,3,chr10,105763461,105765220,-,NaN,NaN,NaN
2,chr10_105765291_105765964_-,ctrl_chx,12,3,chr10,105765291,105765964,-,NaN,NaN,NaN
3,chr10_105779681_105793340_-,ctrl,3,3,chr10,105779681,105793340,-,NaN,NaN,NaN
4,chr10_105793413_105794362_-,ctrl,3,3,chr10,105793413,105794362,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
157679,chrY_90793417_90816349_+,ctrl,332,3,chrY,90793417,90816349,+,chrY_90793417_+,90793417,187.0
157679,chrY_90793417_90816349_+,ctrl,332,3,chrY,90793417,90816349,+,chrY_90816349_+,90816349,7.0
157680,chrY_90793417_90816349_+,ctrl_chx,767,3,chrY,90793417,90816349,+,chrY_90793417_+,90793417,306.0
157681,chrY_90793417_90822286_+,ctrl_chx,3,1,chrY,90793417,90822286,+,chrY_90793417_+,90793417,306.0


Save this table with all types of reads

In [7]:
data_merged_expanded.to_csv(path/'merged_IPSA_j6_s6.csv', index=False)

For futher analysis use only J6 reads

In [8]:
df = data_merged_expanded.drop_duplicates()
df = df.iloc[:,:8].reset_index(drop=True).drop_duplicates()

Make table with cassete exons

In [9]:
cassete_exons = get_cassete_exons_from_ipsa(df).drop_duplicates()

for col in ['start', 'end', 'start_exon', 'end_exon']:
    cassete_exons[col] = cassete_exons[col].astype(str).astype(int)

cassete_exons['chx'] = np.where(
    cassete_exons['experiment'].str.contains('chx', na=False),
    '+chx', '-chx'
)

mask = cassete_exons['chx'] == '+chx'
cassete_exons.loc[mask, 'experiment'] = cassete_exons.loc[mask, 'experiment'].str.replace('_chx', '', regex=False)


We want to expand the table so that one row contains data for both types of experiments

In [10]:
# Grouping by ‘start’ and ‘end’
df_grouped = cassete_exons.pivot_table(index=["start_exon", "end_exon", "experiment"], columns="chx", aggfunc="first")

df_grouped.columns = [f"{col[1]}_{col[0]}" for col in df_grouped.columns]
df_grouped.reset_index(inplace=True)

Convert this table, if there is NA somewhere, fill with zeros (these reids are not present)

In [11]:
df_grouped.fillna(0, inplace=True)

columns = ['chrom', 'strand', 'start', 'end']
for col in columns:
    df_grouped[col] = df_grouped[f'+chx_{col}'].where(df_grouped[f'+chx_{col}'] != 0, df_grouped[f'-chx_{col}'])

to_drop = [f'{sign}chx_{col}' for sign in ('+', '-') for col in columns]
df_grouped.drop(columns=to_drop, inplace=True)

Save table with cassete exons

In [12]:
df_grouped.to_csv(path_out_csv, index=False)

Make custom track for UCSC genome browser

In [13]:
ce_bed = make_track(df_grouped)
ce_bed["score"] = ce_bed["score"].fillna(0)
ce_bed['chromStart'] = ce_bed['chromStart'].astype(int).astype(str)
ce_bed['chromEnd'] = ce_bed['chromEnd'].astype(int).astype(str)

# Writing to BED file
header = ('track name="chx_cassette" description="chx_cassette_exons, PSI = (2e) / (2e + i1 + i2)" '
          'visibility=2 color=103,137,33 itemRgb="on" useScore=1') # Check header for custom track
with open(path_out_bed, "w") as f:
    f.write(header + "\n")
ce_bed.to_csv(path_out_bed, mode="a", sep=' ', index=False, header=False, quoting=3)